In [3]:
import sys
sys.path.insert(0, '..')
sys.path.insert(0, 'plots')

import os

from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets


import json
from SplunkIntelOptimized import SplunkIntelOptimized
from sources.SplunkDatasetNew import SplunkDatasetNew
from sources.SplunkFileSource import SplunkFileSource

import plotly as py
import pandas as pd
import numpy as np
import plotNew as plot

py.offline.init_notebook_mode()
pd.set_option('display.notebook_repr_html', True)
    
data_sources_names = []

data_source_picker = None
time_range_picker = None
event_picker = None
event_text_area = None 
threshold_picker = None
new_time_range_picker = None


#create widgets
def create_widgets():
    global data_source_picker, time_range_picker, event_picker
    global event_text_area, threshold_picker, new_time_range_picker
    for file in os.listdir("../../data_prod"):
            if file.endswith(".json"):
                data_sources_names.append(file)

    data_source_picker = widgets.Dropdown(
            options=data_sources_names,
            description='Data Source:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )    

    time_range_picker = widgets.IntRangeSlider(
                value=[1, 2],
                min=0,
                max=100,
                step=1,
                description='Time:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='i',
                slider_color='white',
                color='black'
        )
    
    threshold_picker = widgets.FloatSlider(
                value=0.8,
                min=0,
                max=1,
                step=0.1,
                description='Threshold:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
                slider_color='white',
                color='black'
        )
    
    event_picker = widgets.Dropdown(
            options=[1,2,3],
            description='Event Number:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )
    
    l = widgets.Layout(height='40px', width='800px')
    event_text_area = widgets.Textarea(value='TA: height=40px', layout=l)

    
def initialize(change):    
    # Just to set the widgets
    all_events = []
    all_events.extend(SplunkFileSource.load_prod_data('../../data_prod/' + data_source_picker.value))    
    minutes = 0
    for dict in all_events:
        if int(dict.get('logCollectionMinute')) > minutes:
            minutes = dict.get('logCollectionMinute')

    time_range_picker.value = [0,minutes]
    time_range_picker.min = 0
    time_range_picker.max = minutes

    
    change_event_handler(None)
    
    
splunkDataset = SplunkDatasetNew()
splunkIntelargs = []

def event_picker_change_handler(change):
    event_text_area.value = json.dumps(splunkDataset.get_all_events_for_notebook()[event_picker.value])

def split(input, length, size):
    input.replace('\n', ' ')
    input.replace('\tat', ' ')
    return '<br>'.join([input[start:start + size] for start in range(0, length, size)])

def get_tool_tips(all_events):
    tooltips = []
    for idx, event in enumerate(all_events):
        tooltips.append(
            split(event[0], min(100, len(event[0])), 100) + '<br> id = ' + str(idx) + '<br> cluster = ' + str(
            event[3]))
            
    return tooltips
    
# Handle data source change
def change_event_handler(change):
    global splunkDataset
    
    control_start = time_range_picker.value[0]
    test_start = time_range_picker.value[0]
    
    control_end = time_range_picker.value[1]
    test_end = time_range_picker.value[1]

    clear_output()

    prev_out_file = None
    while(control_start <= control_end or test_start < test_end):

        del splunkIntelargs[:]
        splunkDataset = SplunkDatasetNew() 

        print(control_start, control_start)
        print(test_start, test_start)
        splunkDataset.load_prod_file('../../data_prod/' + data_source_picker.value,
                                     [control_start, control_start],
                                     [test_start, test_start], 
                                     ['ip-172-31-12-78', 'ip-172-31-15-177', 'ip-172-31-2-144', 'ip-172-31-12-51'],
                                     ['ip-172-31-4-253'],     
                                          prev_out_file)
    
        splunkIntelargs.append('--sim_threshold')
        splunkIntelargs.append(str(float(threshold_picker.value)))
    
        print(splunkIntelargs)

        splunkIntel = SplunkIntelOptimized(splunkDataset, SplunkIntelOptimized.parse(splunkIntelargs))
        splunkDataset = splunkIntel.run()
    
        file_object  = open("result.json", "w")
        file_object.write(splunkDataset.get_output_for_notebook_as_json)
        file_object.close()
        
        prev_out_file = './result.json'
        
        control_start = control_start + 1
        test_start = test_start + 1

    xy_matrix, tooltips, labels,sizes = splunkDataset.control_scatter_plot()
    plot.scatter_plot_groups(xy_matrix, labels, tooltips, ['control', 'test', 'test-anomaly', 'test-unx-freq'], ['blue','green','red', 'orange'])
    
    #xy_matrix, tooltips, labels, sizes = splunkDataset.count_scatter_plot()
    
    #new_sizes = []
    #for size in sizes:
     #   new_sizes.append((3 + len(str(size)))**2)
        
    #print(new_sizes)    
    
    #plot.scatter_plot_groups(xy_matrix, labels, tooltips, ['test', 'test-anomaly'], ['green','red'], new_sizes)


    #for key,data in splunkDataset.get_anom_clusters().items():
     #   for host,anom in data.items():
     #       print(host, anom.get('text'))
    
    xy_matrix, tooltips, labels, clusters = splunkDataset.control_scatter_plot_4d()
    plot.scatter_plot_groups_4d(xy_matrix, labels, clusters, tooltips, ['blue','orange','red'])
    
    hist_data = splunkDataset.count_hist_plot()
    for key,value in hist_data.items():
        plot.hist_plot(value.get('control'), value.get('test'))
            
    event_picker.options = [ x for x in range(len(splunkDataset.get_all_events_for_notebook()))] 
     

create_widgets()

#intialize
initialize(None)
    
#setup even handlers    
data_source_picker.observe(initialize, names="value")
time_range_picker.observe(change_event_handler, names="value")
event_picker.observe(event_picker_change_handler, names="value")
threshold_picker.observe(change_event_handler, names="value")



# show widgets
display(widgets.HBox([data_source_picker]))
display(widgets.HBox([time_range_picker, threshold_picker]))
display(widgets.HBox([event_picker, event_text_area]))

2017-07-18 15:15:20,599 INFO loading file ../../data_prod/splunkLogs.json
2017-07-18 15:15:20,698 INFO Running using file source
2017-07-18 15:15:20,699 INFO Start vectorization....
2017-07-18 15:15:20,701 INFO setting min_df = 1 and max_df = 1.0


(0, 0)
(0, 0)
([0, 0], [0, 0], ['ip-172-31-12-78', 'ip-172-31-15-177', 'ip-172-31-2-144', 'ip-172-31-12-51'], ['ip-172-31-4-253'])
['--sim_threshold', '0.8']
('control count = ', 15)


2017-07-18 15:15:20,880 INFO Running kemans with k = 8
2017-07-18 15:15:21,079 INFO Running kemans with k = 12
2017-07-18 15:15:21,381 INFO Running kemans with k = 14
2017-07-18 15:15:21,730 INFO Running kemans with k = 15
2017-07-18 15:15:22,117 INFO found k = 15


('test count = ', 1)


2017-07-18 15:15:22,426 INFO Detect Count Anomalies....
2017-07-18 15:15:22,428 INFO Using ZeroDeviationClassifier for cluster 6
2017-07-18 15:15:22,430 INFO done
2017-07-18 15:15:22,436 INFO loading file ../../data_prod/splunkLogs.json
2017-07-18 15:15:22,540 INFO loading file ./result.json
2017-07-18 15:15:22,556 INFO Running using file source
2017-07-18 15:15:22,558 INFO Start vectorization....
2017-07-18 15:15:22,560 INFO setting min_df = 1 and max_df = 1.0


...

['--sim_threshold', '0.8']
('control count = ', 30)


2017-07-18 15:16:13,353 INFO Running kemans with k = 15
2017-07-18 15:16:13,733 INFO found k = 15
2017-07-18 15:16:13,734 INFO Running kemans with k = 7
2017-07-18 15:16:13,939 INFO Running kemans with k = 11
2017-07-18 15:16:14,222 INFO Running kemans with k = 13
2017-07-18 15:16:14,574 INFO Running kemans with k = 14


('test count = ', 50)


2017-07-18 15:16:17,276 INFO Running kemans with k = 10
2017-07-18 15:16:17,516 INFO found k = 10
2017-07-18 15:16:17,518 INFO Running kemans with k = 5
2017-07-18 15:16:17,649 INFO found k = 5
2017-07-18 15:16:17,651 INFO Running kemans with k = 2
2017-07-18 15:16:17,706 INFO found k = 2
2017-07-18 15:16:17,707 INFO Running kemans with k = 1
2017-07-18 15:16:17,752 INFO Detect Count Anomalies....
2017-07-18 15:16:17,754 INFO Using ThreeSigmaClassifier for cluster 0
2017-07-18 15:16:17,756 INFO Using ThreeSigmaClassifier for cluster 1
2017-07-18 15:16:17,759 INFO Using ThreeSigmaClassifier for cluster 2
2017-07-18 15:16:17,763 INFO Using ThreeSigmaClassifier for cluster 3
2017-07-18 15:16:17,766 INFO Using ThreeSigmaClassifier for cluster 4
2017-07-18 15:16:17,770 INFO Using ThreeSigmaClassifier for cluster 5
2017-07-18 15:16:17,772 INFO Using ThreeSigmaClassifier for cluster 6
2017-07-18 15:16:17,775 INFO Using ThreeSigmaClassifier for cluster 7
2017-07-18 15:16:17,777 INFO Using Thre

[50, 50, 65, 64, 64, 31, 64, 52, 49, 51, 51, 52, 52, 51, 51, 51, 51, 65, 64, 64, 31, 64, 52, 48, 51, 52, 51, 52, 50, 52, 51, 51, 65, 64, 64, 31, 64, 52, 48, 51, 52, 51, 52, 50, 52, 50, 50, 65, 63, 64, 31, 64, 52, 49, 51, 51, 52, 52, 50, 51]
[135 132  96 129 138 135 135 138 132 135]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[31, 64, 64, 65, 64, 52, 50, 48, 52, 51, 52, 50, 52, 51, 52, 31, 64, 64, 65, 64, 52, 51, 49, 51, 51, 52, 50, 52, 50, 51, 31, 64, 64, 65, 64, 52, 51, 49, 51, 52, 51, 50, 52, 50, 51, 30, 64, 63, 65, 64, 52, 50, 48, 52, 51, 52, 50, 52, 51, 52]
[ 96   3 135 126 138 135 135 135 138 135 135]
[-1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[51, 30, 63, 63, 65, 64, 51, 48, 52, 52, 51, 51, 51, 50, 52, 52, 30, 64, 63, 65, 64, 50, 48, 52, 51, 51, 51, 51, 51, 52, 52, 30, 64, 63, 66, 64, 50, 48, 52, 51, 52, 51, 51, 51, 52, 51, 31, 64, 64, 65, 65, 51, 48, 52, 51, 51, 51, 51, 51, 52]
[ 96 135 129 138 138 138 132 135 135 138]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
